%   Parameters may be provided as name, value pairs, as in
%   [d,p] = ct_dataproc(fname, name1, value1, name2, value2, ...)
%   
%   savename    - cell array of names under which to save processed data
%   name        - cell array of names of channels ({'c1','c2'})
%   ind         - cell array in indices from valcube to use for each
%                channel ({[1,2], 4})
%   fun         - cell array of functions to apply to each channel
%   tsamp       - sampling period (not actually used)
%   pwrat       - power ratio for FRET channel conversion
%   gapmax      - maximum number of bad time points to bridge
%   dlegnthmin  - minimum length of an acceptable trace
%   startonly   - TRUE to only keep tracks that begin at the start of imaging
%   packascell  - TRUE to return data with NaNs removed as a cell array
%                (depricated), FALSE will return each channel as a
%                NaN-padded  cells x time array
%   keeplinfo   - Set to FALSE to discard Lineage info (default: TRUE)
%   verbose     - TRUE to show all warnings

In [ ]:
import re
import warnings
import numpy as np
import os
%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/ct_trimdata.ipynb'
%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/GMD.py'
#%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/iman_powerratio_est.ipynb'
%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/match.ipynb'

In [ ]:
def subf_deffuns(n, ind, pwrat, pfName=None):
    rn = ['kar', 'ktr', 'cpgfp', 'fod']  # Recognized names
    # Descriptors
    nDesc = ['FRET-based Kinase Activity Reporter',
             'Kinase Translocation Reporter',
             'Circularly Permuted GFP Reporter',
             'FRET Over Donor Reporter']
    # Flags indicating if names require pwrat
    needsPWR = [True, False, True, True]

    if pfName is None:
        # Search for pre-defined names
        rmat = [i+1 for i, r in enumerate(rn) if re.search(r, n, re.I)]
    else:
        rmat = [i+1 for i, r in enumerate(rn) if r.lower() == pfName.lower()]

    # Check for inconsistent matches
    nr = len(rmat)
    dtxt = 'Applying no processing function.'
    if nr < 1:
        rmat = [0]
        warnings.warn(f'Name "{n}" not recognized. {dtxt}')
    elif nr > 2:
        rmat = [0]
        warnings.warn(f'Unparsable name, "{n}". {dtxt}')

    # Check that indices are provided
    nInd = len(ind)
    if nInd < 1:
        rmat = [0]
        warnings.warn(f'No Channel index found for ("{n}"). {dtxt}')

    # Check for power ratio, as needed
    if rmat[0] > 0 and needsPWR[rmat[0]] and (n not in pwrat or pwrat[n] is None):
        warnings.warn(f'Power ratio must be supplied for a {nDesc[rmat[0]]} ("{n}"). {dtxt}')
        hasPWR = False
    else:
        hasPWR = True

    # Define desired processing function
    sfun = None  # Start with an empty placeholder
    if rmat[0] == 1:
        if hasPWR:
            if nInd == 1:
                sfun = lambda x1: 1 - x1 / pwrat[n]
                print('KAR 1')
            elif nInd == 2:
                sfun = lambda x1, x2: 1 - (x1 / x2) / pwrat[n]
                print('KAR 2')
            else:
                warnings.warn(f'Too many indices provided for a {nDesc[rmat]}. This will result in error.')
    elif rmat[0] == 2:
        if nInd == 1:
            sfun = lambda x1: x1
            warnings.warn(f'KTR "{n}" provided only one channel index (ok if input channel is a precomputed ratio). {dtxt}')
        elif nInd == 2:
            sfun = lambda x1, x2: x1 / x2
            print('KTR 2')
        else:
            warnings.warn(f'Too many indices provided for a {nDesc[rmat]}. This will result in error.')
    elif rmat[0] == 3:
        if hasPWR:
            if nInd == 1:
                sfun = lambda x1: x1
                warnings.warn(f'CP-GFP "{n}" provided only one channel index (ok if input channel is a precomputed ratio). {dtxt}')
            elif nInd == 2:
                sfun = lambda x1, x2: (x1 / x2) / pwrat[n]
            else:
                warnings.warn(f'Too many indices provided for a {nDesc[rmat]}. This will result in error.')
    elif rmat[0] == 4:
        if hasPWR:
            if nInd == 1:
                sfun = lambda x1: x1
                warnings.warn(f'FRET-Over-Donor Reporter "{n}" provided only one channel index (ok if input channel is a precomputed ratio). {dtxt}')
            elif nInd == 2:
                sfun = lambda x1, x2: (x1 / x2) / pwrat[n]
            else:
                warnings.warn(f'Too many indices provided for a {nDesc[rmat]}. This will result in error.')

    # IF sfun was not assigned, provide the default 1 argument passthrough
    if sfun is None:
        sfun = lambda x: x
        print('X')

    return sfun

In [ ]:
def ct_dataproc(**kwargs):
    """
    Pre-processing and input option pair parsing.
    """
#define input
    cyto_path = cyto_dir
    nu_path = nu_dir
    df = match(cyto_path,nu_path)
    df['frame'] = df['frame'].astype(int)
    its = inten_array(df)
# Default parameters
    p = {
        'savename': None,
        'name': ['ekar', 'fra', 'x', 'y'],
        'ind': [[4, 5], 3, 7, 8],
        'fun': [None],
        'tsamp': 1,
        'pwrat': None,
        'stripidx': False,
        'verbose': True,
        'gapmax': 2,
        'dlengthmin': 100,
        'startonly': False,
        'keeplinfo': True,
    }

    # Input option pair parsing
    for key, value in kwargs.items():
        p[key.lower()] = value

    # Block warnings if not running verbose
    if not p['verbose']:
        warnings.filterwarnings('ignore')

    # Check parameter structure
    if isinstance(p['name'], str):
        p['name'] = [p['name']]
    nn = len(p['name'])
    if p['fun'] is not None and len(p['fun']) != nn:
        p['fun'] += [None] * (nn - len(p['fun']))
    if isinstance(p['ind'], int) or isinstance(p['ind'], list) and all(isinstance(i, int) for i in p['ind']):
        if nn == 1:
            p['ind'] = [p['ind']]
        elif len(p['ind']) == nn:
            p['ind'] = [[i] for i in p['ind']]
        else:
            raise ValueError('Number of Names (name) and Indices (ind) must match')

    # Check power ratio definition
    if not isinstance(p['pwrat'], dict):
        if p['pwrat'] is None:
            p['pwrat'] = {}
        elif isinstance(p['pwrat'], list) and len(p['pwrat']) == len(p['name']):
            p['pwrat'] = {name: val for name, val in zip(p['name'], p['pwrat'])}
        elif isinstance(p['pwrat'], int) or isinstance(p['pwrat'], float):
            p['pwrat'] = {name: p['pwrat'] for name in p['name']}
        else:
            raise ValueError('Invalid input: pwrat must be a dictionary with keys matching output channel names (the "name" input), each containing the associated power-ratio value. It may also be a numeric list of power-ratio values, either scalar or the length of output channel names.')
# Refine data

    # Load data - check first if lineage info is present
    haslinfo = any(index == 'linfo' for index in df.columns) and p['keeplinfo']

    # Extract desired channels prior to cleanup
    pd = {}
    for sc in range(nn):
        if not p['fun'][sc]:
            p['fun'][sc] = subf_deffuns(p['name'][sc], p['ind'][sc], p['pwrat'])
        elif isinstance(p['fun'][sc], str):
            p['fun'][sc] = subf_deffuns(p['name'][sc], p['ind'][sc], p['pwrat'], p['fun'][sc])
        intemp = [its[:,:,i] for i in p['ind'][sc]]
        # Apply processing function and retain result
        pd[p['name'][sc]] = p['fun'][sc](*intemp)

    # Run data trimming procedure
    d = ct_trimdata(pd, p)
    # IF linfo is present, apply trimming index to linfo, append to output
    if haslinfo and d['cellindex'] and len(td['linfo']) >= max(d['cellindex']):
        # Filter linfo
        linfo = td['linfo'][d['cellindex'],:]
        # Build map to new indices
        imap = np.full(d['cellindex'][-1], np.nan)
        imap[d['cellindex']] = np.arange(np.count_nonzero(d['cellindex']))
        # Adjust linfo references
        uniq_linfo = np.unique(linfo[~np.isnan(linfo)])
        for sl in uniq_linfo:
            linfo[linfo == sl] = imap[int(sl)]
        # Append linfo to the output
        d['linfo'] = linfo


    return df,d,p


# Uasge ERKTR
#params
#FRET channels
fc = []
#State sampling interval
tsamp = 6
#Declare (new) output channel names
cn = ['ERKTR']
#Declare matching input channel indices (or descriptors)
#for ratios list numerator (or FRET Donor) first, x/y => [x,y]
ci = [[6,2]]
#Set minimum track length
minL = 25
#Set maximum gap size
maxGap = 2
#Set 'start only' flag
sto = False


#Check if 'fc' exists and is not empty
if 'fc' in locals() and fc:  
    GMD = mdi
    # Calculate the power ratio for FRET signals
    prat = iman_powerratio_est(GMD, None, fc)
    prat = list(prat.astype(float))


#Usage difference: take one series as input, not multiple
d,p = ct_dataproc(df,
    tsamp = tsamp,
    name = cn,
    ind = ci,
    pwrat = prat,
    dlengthmin = minL,
    gapmax = maxGap,
    startonly = sto)


# Uasge EKAR for Nu
#FRET channels
fc = ['cfp','yfp']
#State sampling interval
tsamp = 6
#Declare (new) output channel names
cn = ['EKAR']
#Declare matching input channel indices (or descriptors)
#for ratios list numerator (or FRET Donor) first, x/y => [x,y]
ci = [[5,6]]
#Set minimum track length
minL = 25
#Set maximum gap size
maxGap = 2
#Set 'start only' flag
sto = False
prat = []

#Check if 'fc' exists and is not empty
if 'fc' in locals() and fc:  
    GMD = mdi
    # Calculate the power ratio for FRET signals
    prat = iman_powerratio_est(GMD, None, fc)
    prat = list(prat.astype(float))


#Usage difference: take one series as input, not multiple
d,p = ct_dataproc(df,
    tsamp = tsamp,
    name = cn,
    ind = ci,
    pwrat = prat,
    dlengthmin = minL,
    gapmax = maxGap,
    startonly = sto)
